In [1]:
import pandas as pd

df = pd.read_feather('../../../data/data_processed/us_videos_ads.feather')
start_urls = df['ad_url'].tolist()

df.head()

,ad_id,date_range_start,ad_url,advertiser_id,advertiser_name
0,CR17327428454810910721,2018-05-31,https://adstransparency.google.com/advertiser/...,AR02433910412765298689,"CLEARPATH ACTION FUND, INC."
1,CR18391121292288851969,2018-05-31,https://adstransparency.google.com/advertiser/...,AR02838733826560098305,DIANE BLACK FOR GOVERNOR
2,CR17941042804528513025,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE
3,CR16806135698431148033,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE
4,CR06904690442680401921,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE


In [2]:
df = pd.read_parquet('../../../data/data_processed/us_videos_ads.parquet')
start_urls = df['ad_url'].tolist()

df.head()

,ad_id,date_range_start,ad_url,advertiser_id,advertiser_name
0,CR17327428454810910721,2018-05-31,https://adstransparency.google.com/advertiser/...,AR02433910412765298689,"CLEARPATH ACTION FUND, INC."
1,CR18391121292288851969,2018-05-31,https://adstransparency.google.com/advertiser/...,AR02838733826560098305,DIANE BLACK FOR GOVERNOR
2,CR17941042804528513025,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE
3,CR16806135698431148033,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE
4,CR06904690442680401921,2018-05-31,https://adstransparency.google.com/advertiser/...,AR07102652013703331841,JOSH HAWLEY FOR SENATE


In [4]:
start_urls[0]

'https://adstransparency.google.com/advertiser/AR02433910412765298689/creative/CR17327428454810910721?political=&region=US'

In [8]:
sample = df.sample(n=16)
sample.to_parquet('../../../data/data_processed/us_video_ads_sample.parquet')

In [1]:
%load_ext autoreload
%autoreload 2

import json
import csv
import pandas as pd
from urllib.parse import urlparse, parse_qs

from scrapy.http import Request
import os
import scrapy


class AdspiderSpider(scrapy.Spider):
    name = "adspider"
    allowed_domains = ["adstransparency.google.com"]

    # print(os.getcwd())
    # print(os.listdir('../../../data/data_processed/'))
    # Read the start_urls from the us_video_rows.fst file
    df = pd.read_parquet('../../../data/data_processed/us_video_ads_sample.parquet')
    start_urls = df['ad_url'].tolist()

    # Set batch size to 8
    batch_size = 8

    headers = {
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded",
        "Pragma": "no-cache",
        "Sec-Ch-Ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": "\"Windows\"",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "X-Framework-Xsrf-Token": "",
        "X-Same-Domain": "1",
    }

    def start_requests(self):
        for i in range(0, len(self.start_urls), self.batch_size):
            batch_urls = self.start_urls[i:i+self.batch_size]
            for url in batch_urls:
                # Parse the URL to extract the AR* and CR* values
                parsed_url = urlparse(url)
                path_parts = parsed_url.path.split('/')
                ar_value = path_parts[2]
                cr_value = path_parts[4]

                # Generate the body of the POST request
                body = f'f.req=%7B%221%22%3A%22{ar_value}%22%2C%222%22%3A%22{cr_value}%22%2C%225%22%3A%7B%221%22%3A2%7D%7D'

                yield Request(
                    url="https://adstransparency.google.com/anji/_/rpc/LookupService/GetCreativeById?authuser=",
                    callback=self.parse,
                    method='POST',
                    headers=self.headers,
                    body=body,
                    cb_kwargs=dict(batch_index=i, url_index=self.start_urls.index(url))
                )

    def parse(self, response, batch_index, url_index):
        data = json.loads(response.text)

        try:
            # Extract the YouTube link from the JSON data
            youtube_link = data['1']['5'][0]['2']['4']
            # Add the youtube_link to the corresponding row in the original DataFrame
            self.df.loc[url_index, 'youtube_link'] = youtube_link
        except:
            # If there is an error, write 'NoVideo' in the DataFrame
            self.df.loc[url_index, 'youtube_link'] = 'NoVideo'

        # Write the processed URLs to a CSV file in batches of 8 rows
        self.df.to_csv(f'processed_urls_{batch_index}.csv', index=False)

from scrapy.crawler import CrawlerProcess

# Create a CrawlerProcess instance with some settings
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': 'ERROR',
    'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7'
})

# Add your spider class to the CrawlerProcess
process.crawl(AdspiderSpider)

# Start the crawling process
process.start()